# Integrate a Custom Weather Tool in Azure AI Agents

This notebook demonstrates how to expose **custom Python functions** as callable **tools** to an **Azure AI Agent**, and how the agent can **auto‑invoke** those tools to answer user questions.

1. Load configuration from environment variables.
2. Connect to `AIProjectClient` and `AgentsClient`.
3. Define custom functions (weather + time).
4. Register them via `FunctionTool` and a `ToolSet`.
5. Create an agent, start a thread, and chat interactively.

> **Prerequisites**
> * Azure credentials available to `DefaultAzureCredential` (e.g., Azure CLI login).
> * Environment variables: `PROJECT_ENDPOINT`, `MODEL_DEPLOYMENT_NAME`.
> * Python packages: `azure-identity`, `azure-ai-projects`, `azure-ai-agents`, `python-dotenv`.


## 1) Imports and setup
Minimal imports for Azure Agents, environment loading, and helper utilities.
- `DefaultAzureCredential` authenticates using your local Azure identity.
- `AIProjectClient` / `AgentsClient` establish project and agent connections.
- Standard libs support the tool implementations.


In [2]:
import os
from dotenv import load_dotenv
from datetime import datetime as dt, timezone
from random import randint
from typing import Set, Callable, Any

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FunctionTool, ToolSet, MessageRole

## 2) Load environment variables
We read configuration from a local `.env` file so secrets aren’t hard‑coded.
- `PROJECT_ENDPOINT`: your Azure AI project endpoint (e.g., `https://<region>.models.ai.azure.com`).
- `MODEL_DEPLOYMENT_NAME`: name of the deployed chat model to back this agent.


In [5]:
# Load Azure configuration values (endpoint, model name)
load_dotenv()

endpoint = os.environ.get('PROJECT_ENDPOINT')
model = os.environ.get('MODEL_DEPLOYMENT_NAME')

assert endpoint, 'PROJECT_ENDPOINT is not set. Please add it to your .env file.'
assert model, 'MODEL_DEPLOYMENT_NAME is not set. Please add it to your .env file.'

#print('Project endpoint:', endpoint)
#print('Model deployment name:', model)

## 3) Connect to Azure AI project and agents
We create two clients:
- `AIProjectClient`: project context and OpenAI‑compatible client access.
- `AgentsClient`: manage agents, threads, messages, runs, and tools.


In [6]:
# Instantiate the Azure Project client
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

# Optional: get the OpenAI-compatible chat client if needed elsewhere
chat_client = project_client.get_openai_client()

# Instantiate the Agents client for managing agents and tool calls
agent_client = AgentsClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

print('Clients are ready.')


Clients are ready.


## 4) Define custom tool functions
These Python functions will be exposed to the agent as callable tools.
- `get_weather(location: str) -> str`: simulates a weather lookup and returns a friendly sentence.
- `get_time() -> str`: returns the current UTC time as a formatted string.

> Note: These are *simulations* for demo purposes. Replace with real API calls as needed.


In [7]:
def get_weather(location: str) -> str:
    """
    Simulate a weather lookup for a given city.
    Randomly generates conditions and a high temperature.
    """
    conditions = ['sunny', 'cloudy', 'rainy', 'stormy', 'snowing', 'hot']
    temp = randint(0, 30)
    return f"The weather in {location} is {conditions[randint(0,5)]} with a high temperature of {temp}°C."


def get_time() -> str:
    """
    Return the current timestamp in UTC.
    """
    current_time = dt.now(tz=timezone.utc)
    return f"The current UTC time is {current_time.strftime('%Y-%m-%d %H:%M:%S')}."


## 5) Register tools (FunctionTool + ToolSet)
Azure Agents accepts a **set of callable functions** when constructing `FunctionTool`.
We then add that tool bundle to a `ToolSet` and enable auto function calls.


In [8]:
# Collect your custom functions in a set (required by Azure's FunctionTool signature)
user_functions: Set[Callable[..., Any]] = {
    get_weather,
    get_time,
}

# Wrap the functions into a FunctionTool and add to a ToolSet
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

# Allow the agent runtime to automatically call these tools during a run
agent_client.enable_auto_function_calls(tools=toolset)
print('Tools registered and auto-calling enabled.')


Tools registered and auto-calling enabled.


## 6) Create the agent
The agent is configured with:
- a backing `model` (deployment name),
- a friendly `name`,
- short `instructions`, and
- the `toolset` with our custom tools.


In [9]:
agent = agent_client.create_agent(
    model=model,
    name='weather-reporter',
    instructions='You are a helpful assistant that can provide weather and time information.',
    toolset=toolset,
)
print('Agent created:', agent.id)


Agent created: asst_Bs8DDKMVf24P7fZ7LQrOCG5a


## 7) Start a thread (conversation)
Each conversation is represented by a thread. Messages and tool calls are tied to this thread ID.


In [10]:
thread = agent_client.threads.create()
print('Thread created:', thread.id)


Thread created: thread_Y3BUXV83mY7W5Uf3A7vyN0XB


## 8) Quick test: single prompt
Send one message that lets the agent decide which tools to call.
The agent should call `get_weather` and/or `get_time` automatically.


In [12]:
# Send a single test prompt
message = agent_client.messages.create(
    thread_id=thread.id,
    role='user',
    content="What's the weather in Stuttgart and what is the current UTC time?"
)

# Process the run (the agent may auto-call our tools)
run = agent_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id
)

if run.status == 'failed':
    raise RuntimeError(f"Run failed: {run.last_error}")

# Retrieve the last assistant message
last_msg = agent_client.messages.get_last_message_text_by_role(
    thread_id=thread.id,
    role=MessageRole.AGENT
)

if last_msg:
    print('Response from Agent:', last_msg.text.value)
else:
    print('No response was returned. Check tool registration and agent configuration.')


Response from Agent: The weather in Stuttgart is rainy with a high temperature of 16°C.  
The current UTC time is 09:31:50 on February 6, 2026.


## 9) Interactive chat loop (optional)
Run this cell to chat with the agent. Type `quit` to end the session.


In [13]:
# Uncomment to enable interactive mode in a notebook / terminal environment.
# while True:
#     user_prompt = input('Enter a location or command (type 'quit' to exit): ')
#     if user_prompt.lower() == 'quit':
#         break
#     if not user_prompt.strip():
#         continue
#
#     agent_client.messages.create(
#         thread_id=thread.id,
#         role='user',
#         content=user_prompt,
#     )
#
#     run = agent_client.runs.create_and_process(
#         thread_id=thread.id,
#         agent_id=agent.id
#     )
#
#     if run.status == 'failed':
#         print(f"Run failed: {run.last_error}")
#         continue
#
#     last_msg = agent_client.messages.get_last_message_text_by_role(
#         thread_id=thread.id,
#         role=MessageRole.AGENT
#     )
#     if last_msg:
#         print('Response from Agent:', last_msg.text.value)


---
**Notes**
- Replace the simulated weather with a real API if needed (e.g., call a weather service).
- Ensure the model deployment used for `model` supports tool/function calling.
- If you face authentication issues, verify your Azure CLI login and subscription context.
